In [1]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
from mce.core.climate_parms import ParmEstimate

In [2]:
# csv output introduces slight differences and affects optimization results

# df_cmip_norm = (
#     pd.read_csv('data/cmip5_normalized_1.csv', index_col=[0, 1, 2])
#     .rename(columns=float)
# )
df_cmip_norm = pd.read_pickle('data/cmip5_normalized_1.pkl')

In [3]:
nl = 3

if nl == 3:
    args = ([1., 1.], [1., 10., 20.])
elif nl == 2:
    args = ([1.], [2., 100.])

kw = {'lamb': 1., 'ts4xeq': 7., 'beta': 1.}

df_parms = {}

for source_id, v in df_cmip_norm.groupby(level=0):
    df = v.droplevel(0)
    time = np.arange(150) + 0.5
    data_gcm = [
            df.loc[('rtnt', 'abrupt4xCO2')].dropna().values,
            df.loc[('tas', 'abrupt4xCO2')].dropna().values,
            df.loc[('rtnt', '1pctCO2')].dropna().values,
            df.loc[('tas', '1pctCO2')].dropna().values,
        ]
    if [len(d1) for d1 in data_gcm] != [150, 150, 140, 140]:
        raise ValueError('Invalid data length')

    data_std = [
        df.loc[('rtnt', 'piControl')].dropna().std(ddof=0),
        df.loc[('tas', 'piControl')].dropna().std(ddof=0),
    ]

    obj = ParmEstimate(nl=nl)
    px = obj.initpars(*args, **kw)
    alpha, beta, lamb, asj, tauj = obj.minimize_wrap(time, data_gcm, data_std)

    ecs = obj.forcing.x2erf(2.) / lamb
    t70 = np.log(2.) / np.log(1.01)
    tcr = ecs * (1 - (asj * tauj * (1 - np.exp(-t70/tauj))).sum() / t70)

    regress = linregress(data_gcm[1], data_gcm[0])
    lambda_reg = -regress.slope
    ecs_reg = 0.5 * regress.intercept / lambda_reg
    tcr_gcm = data_gcm[3][70-10:70+10].mean()

    df_parms[source_id] = pd.Series({
        'ecs': ecs,
        'ecs_reg': ecs_reg,
        'tcr': tcr,
        'tcr_gcm': tcr_gcm,
        'co2_alpha': alpha,
        'co2_beta': beta,
        'lambda': lamb,
        'lambda_reg': lambda_reg,
        **{
            f'tau{i}': value
            for i, value in enumerate(tauj)
        },
        **{
            f'a{i}': value
            for i, value in enumerate(asj)
        },
    })

df_parms = pd.DataFrame(df_parms).T